# netunicorn usage example
This example shows basic client-side usage of netunicorn API.
Prerequisites:
- overall understanding of the project
- deployed netunicorn infrastructure and director services
- known `endpoint`, `login`, and `password` for the connection

To work with the project, you need to install several packages:
- `netunicorn-base` - provides abstractions and classes to create pipelines and define experiments. If you want to just define your pipeline and write tasks, you need only this package.
- `netunicorn-client` - provides connectivity to netunicorn infrastructure. You need this package to submit and execute experiments.
- `netunicorn-library` - a library of predefined and contributed tasks for the platform. You can use tasks in this package for your pipelines, and submit your code here to share. Please note, that most of the tasks there are provided 'as-is' by other teams and developers, and netunicorn team doesn't guarantee their correctness.

In [1]:
%pip install netunicorn-base
%pip install netunicorn-client
%pip install netunicorn-library

Let's import needed classes

In [2]:
# client to connect to the infrastructure
from netunicorn.client.remote import RemoteClient

# basic abstraction for experiment creation and management
from netunicorn.base.experiment import Experiment
from netunicorn.base.pipeline import Pipeline

# task to be executed in the pipeline
# you can write your own tasks, but now let's use simple predefined one
from netunicorn.library.basic import SleepTask

At first, we want to define pipeline to execute. Pipeline consists of tasks located on different stages, and would be executed by each node where it would be assigned later.
To creeate pipeline, instantiate Pipeline object and use command `.then()` to define a new stage. All tasks on the same stage would be started together, and the next stage would start only when all tasks from the current stage successfully finished.

In [3]:
# we will use simple SleepTask for this example
# you can look at the source code of the SleepTaskLinuxImplementation to understand how it works

pipeline = Pipeline()

# Notice, that executor will first in parallel execute `sleep 5` and `sleep 3`...
pipeline = pipeline.then([
    SleepTask(5),
    SleepTask(3)
]).then(
    # ...and after they finished (after 5 second in total) will execute `sleep 10`
    SleepTask(10)
)

You can combine multiple tasks and stages to create your own pipeline. Each instance of a task in a pipeline would be serialized (together with all parameters) and sent to the executor. The result of task (and pipeline in general) execution would be serialized and sent back to you.

Experiment is defined as one or multiple assignments of pipelines to particular nodes. To define the experiment, we should receive available nodes in infrastructure.

To access the infrastructure, you need the next known parameters, provided by netunicorn installation administrators:

In [4]:
# API connection endpoint
endpoint = "<connection endpoint>"

# user login
login = "<your username>"

# user password
password = "<your password>"

In [5]:
# let's create a client with these parameters
client = RemoteClient(endpoint=endpoint, login=login, password=password)

Using client, you can receive information about available nodes, and then filter them and take needed amount for your experiment.

In [6]:
# let's receive all available for us minions...
nodes = client.get_minion_pool()

# ... and take first 3 from the list
nodes = nodes.take(3)

# Each node object is a class with additional properties and information
# You don't have to use it, but you can to better target your experiments and pipelines
print(nodes)
for node in nodes:
    print(f"Name: {node.name}, properties: {node.properties}")

[raspi-e4:5f:01:75:6b:2c, raspi-e4:5f:01:56:d9:a3, raspi-e4:5f:01:72:a2:eb]
Name: raspi-e4:5f:01:75:6b:2c, properties: {'location': '', 'osarch': 'arm64', 'kernel': 'Linux'}
Name: raspi-e4:5f:01:56:d9:a3, properties: {'location': '', 'osarch': 'arm64', 'kernel': 'Linux'}
Name: raspi-e4:5f:01:72:a2:eb, properties: {'location': '', 'osarch': 'arm64', 'kernel': 'Linux'}


For simplicity, our first experiment would consist of 3 nodes running the same pipeline. Let's create Experiment instance and add all 3 nodes with the pipeline using `map()` method. You can read the documentation about other methods of creating assignments (called `Deployments` in netunicorn).

In [7]:
experiment = Experiment().map(nodes, pipeline)

# let's explore experiment object
print(experiment)
print()
for deployment in experiment:
    print(deployment.minion)
    print(deployment.environment_definition)
    print()

<Deployment: Minion=raspi-e4:5f:01:75:6b:2c, executor_id=, prepared=False>; <Deployment: Minion=raspi-e4:5f:01:56:d9:a3, executor_id=, prepared=False>; <Deployment: Minion=raspi-e4:5f:01:72:a2:eb, executor_id=, prepared=False>

raspi-e4:5f:01:75:6b:2c
DockerImage(commands=[], image=None, build_context=BuildContext(python_version='3.8.13', cloudpickle_version='2.1.0'))

raspi-e4:5f:01:56:d9:a3
DockerImage(commands=[], image=None, build_context=BuildContext(python_version='3.8.13', cloudpickle_version='2.1.0'))

raspi-e4:5f:01:72:a2:eb
DockerImage(commands=[], image=None, build_context=BuildContext(python_version='3.8.13', cloudpickle_version='2.1.0'))



To submit the experiment, we need to create a user-wide unique name for the experiment, and call an appropriate method of the `client` object. Notice, that you can submit the same experiment several times with different names to be executed more than once.

In [9]:
experiment_name = "experiment_cool_name"
client.prepare_experiment(experiment, experiment_name)

'experiment_cool_name'

When you submit an experiment, netunicorn services automatically create or download a virtual environment for execution of the pipeline, insert serialized pipeline inside and distribute these environments to the desired nodes.

To check status of the experiment, you can use corresponding method of the `client` object.

In [11]:
# status will change from PREPARING to READY when compiled and deployed
info = client.get_experiment_status(experiment_name)
print(info.status)

ExperimentStatus.READY


In [12]:
# One of the returned objects is a prepared experiment. It holds all the information about deployments compilation
# Some nodes could be failed to prepare due to various reasons
prepared_experiment = info.experiment
for deployment in prepared_experiment:
    print(deployment.minion)
    print(deployment.prepared)
    print(deployment.error)
    print()

raspi-e4:5f:01:75:6b:2c
True
None

raspi-e4:5f:01:56:d9:a3
True
None

raspi-e4:5f:01:72:a2:eb
True
None



When the status is ready, nodes are prepared for execution and downloaded all the needed environments and pipelines (don't forget to check `prepared` status of the returned experiment to confirm).

Now you can ask `client` object to start the experiment. It will ask nodes to spin up executors and will collect the execution results.

In [13]:
client.start_execution(experiment_name)

'experiment_cool_name'

In [14]:
# Again, let's check experiment status until it changes from EXECUTING to FINISHED
info = client.get_experiment_status(experiment_name)
print(info.status)

ExperimentStatus.FINISHED


In [16]:
for report in info.execution_result:
    print(report.error)
    result, log = report.result
    print(result)
    print(log)
    print()

None
<Success: ((<Success: 5>, <Success: 3>), <Success: 10>)>
['Parsed configuration: Gateway located on <redacted>\n', 'Current directory: /\n', 'Pipeline loaded from local file, executing.\n', 'Pipeline finished, start reporting results.\n']

None
<Success: ((<Success: 5>, <Success: 3>), <Success: 10>)>
['Parsed configuration: Gateway located on <redacted>\n', 'Current directory: /\n', 'Pipeline loaded from local file, executing.\n', 'Pipeline finished, start reporting results.\n']

None
<Success: ((<Success: 5>, <Success: 3>), <Success: 10>)>
['Parsed configuration: Gateway located on <redacted>\n', 'Current directory: /\n', 'Pipeline loaded from local file, executing.\n', 'Pipeline finished, start reporting results.\n']



Congrats! We (hope, you too ^_^) successfully finished the basic experiment using the netunicorn platform.
For next steps, you can read the documentation on creating more complex experiments, including writing your own tasks, providing your own Docker containers, experiment synchronization, etc.

For all questions, refer to the official repo: https://github.com/SNL-UCSB/netunicorn and netunicorn team.